In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Experiments: Autologging

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.10.2

## Overview

As part of the data science team, you want to try different modeling approaches during experimentation phase.To guarantee reproducibility, each approach has different parameters that you need to manually track This is a time consuming task. To address this challenge, Vertex AI SDK introduces autologging, a one-line code SDK capability which leverages MLflow to provide automatic metrics and parameters tracking associated with your  Vertex AI Experiments and experiment runs. Learn more about [Autologging data to an experiment run](https://cloud.google.com/vertex-ai/docs/experiments/autolog-data).

### Objective

In this tutorial, you learn how to use `Vertex AI Autologging`.

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI Experiments

The steps performed include:

- Enable autologging in the Vertex AI SDK.
- Train one scikit-learn model
- Train one LightGBM model
- Train three XGBoost models (setting different hyperparameters)
- See all the resulting experiment runs with the evaluation metrics (accuracy, precision, recall, and F1-score) and parameters autologged to Vertex AI Experiments without setting any experiment run.


### Dataset

The dataset is the [UCI Car Evaluation dataset](https://archive-beta.ics.uci.edu/dataset/19/car+evaluation), which is derived from simple hierarchical decision model and it contains attributions to predict car evaluation class.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI Experiments
* Vertex AI Tensorboard
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and [Cloud Storage pricing](https://cloud.google.com/storage/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook.

In [ ]:
# Install the packages
USER = ""
! pip3 install {USER} --upgrade google-cloud-aiplatform tensorflow
! pip3 install {USER} --upgrade pandas scikit-learn category_encoders torch torchdata torchmetrics mlflow==2.4.0 lightgbm
! pip3 install {USER} --upgrade protobuf==3.20.3


### Colab only: Uncomment the following cell to restart the kernel.
Automatically restart kernel after installs so that your environment can access the new packages

In [ ]:
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Setup

### Define env variables

In [ ]:
# Set the project id
ROJECT_ID = "my_project"  # @param {type:"string"}
! gcloud config set project {PROJECT_ID}

# Set your region
REGION = "europe-west4"  # @param {type: "string"}

# Set a bucket to store intermediate artifacts
BUCKET_URI = "gs://my_bucket"  # @param {type:"string"}

12106.47s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


E0619 14:32:15.362065081  127664 backup_poller.cc:136]                 Run client channel backup poller: UNKNOWN:pollset_work {created_time:"2023-06-19T14:32:15.361979318+00:00", children:[UNKNOWN:Bad file descriptor {created_time:"2023-06-19T14:32:15.361929472+00:00", errno:9, os_error:"Bad file descriptor", syscall:"epoll_wait"}]}
Updated property [core/project].


**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
# ! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### Upload dataset to BigQuery, load it to a pandas dataframe

1. Upload the [ UCI Car Evaluation dataset](https://archive-beta.ics.uci.edu/dataset/19/car+evaluation) (mentioned above) to a BigQuery table (**your turn**)
2. Upload data from this table to a pandas dataframe.

(you could also interact with the data in CSV format if you prefer, but then you'd need to do some minor changes to the code)

#### 2. Upload BigQuery table to a pandas dataframe

In [ ]:
%load_ext google.cloud.bigquery

from google.cloud import bigquery
client = bigquery.Client()

## JUST FOR COLAB
#from google.colab import data_table
#data_table.enable_dataframe_formatter()

In [ ]:
%%bigquery cars_df --project $PROJECT_ID

SELECT * FROM `my_project.my_dataset.my_table` # @param {type: "string"}

In [ ]:
cars_df

### Import libraries

Import the Vertex AI SDK to log experiments in Vertex AI Experiments.

In [ ]:
from google.cloud import aiplatform as vertex_ai

### Helper functions

To run experiments it is not uncommon to define experiment helpers, one per each modelling approach you plan to evaluate. Below you define the following experiment helpers:

*   `train_sklearn_model`: A helper function to train a Decision Tree model using Sklearn.
*   `train_xgboost_model`: A helper function to train a Decision Tree model using XGBoost.
*   `train_LGBM_model`: A helper function to train a Decision Tree model using LightGBM.


In [ ]:
def set_seed(seed: int):
    """
    A function to set the seed for reproducibility.
    Args:
        seed: Seed to be set
    Returns:
        None
    """
    import random

    import numpy as np
    import tensorflow as tf

    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)



def train_sklearn_model(df: pd.DataFrame, test_size: int, max_depth: int):
    """
    A function to train a Decision Tree model using sklearn.
    Args:
        df: Input data, which needs to be split into train and test
        test_size: Size of the test set
        max_depth: Maximum depth of the Decision Tree
    Returns:
        None
    """

    # Libraries
    import pandas as pd
    from category_encoders import OrdinalEncoder
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    from sklearn.pipeline import Pipeline
    from sklearn.tree import DecisionTreeClassifier

    # Train, test split
    print("Generating train and test data...")
    x = df[["buying", "maint", "doors", "persons", "lug_boot", "safety"]]
    y = df[["class"]]
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=test_size, shuffle=True
    )

    # Build pipeline
    print("Building pipeline...")
    pipe = Pipeline(
        [
            ("encoder", OrdinalEncoder()),
            ("model", DecisionTreeClassifier(criterion="gini", max_depth=max_depth)),
        ]
    )

    # Train model
    print("Training model...")
    pipe.fit(x_train, y_train)

    # Evaluate model
    print("Evaluating model...")
    y_pred = pipe.predict(x_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print("Accuracy:", round(accuracy, 3))
    precision = metrics.precision_score(y_test, y_pred)
    print("Precision:", round(precision, 3))
    recall = metrics.recall_score(y_test, y_pred)
    print("Recall:", round(recall, 3))
    f1 = metrics.f1_score(y_test, y_pred)
    print("F1 score:", round(f1, 3))


def train_xgboost_model(df: pd.DataFrame, test_size: int, max_depth: int, n_estimators: int,  enable_categorical: bool=True):
    """
    A function to train a Decision Tree model using XGboost.
    Args:
        df: Input data, which needs to be split into train and test
        test_size: Size of the test set
        max_depth: Maximum depth of the Decision Tree
        n_estimators: Number of trees
        enable_categorical: Whether to enable categorical features
    Returns:
        None
    """

    # Libraries
    import pandas as pd
    from xgboost import XGBClassifier
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder


    # Convert categorical columns to numerical values
    for column in cars_df.select_dtypes(include='object').columns:
        label_encoder = LabelEncoder()
        cars_df[column] = label_encoder.fit_transform(cars_df[column])

    # Train, test split
    print("Generating train and test data...")
    x = cars_df[["buying", "maint", "doors", "persons", "lug_boot", "safety"]]
    y = cars_df[["class"]]
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=test_size, shuffle=True
    )

    # Train model
    print("Training model...")
    xgb_clf = XGBClassifier(
        max_depth=max_depth, n_estimators=n_estimators, objective="binary:logistic"
    )
    xgb_clf.fit(x_train, y_train)

    # Evaluate model
    print("Evaluating model...")
    y_pred = xgb_clf.predict(x_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print("Accuracy:", round(accuracy, 3))
    precision = metrics.precision_score(y_test, y_pred)
    print("Precision:", round(precision, 3))
    recall = metrics.recall_score(y_test, y_pred)
    print("Recall:", round(recall, 3))
    f1 = metrics.f1_score(y_test, y_pred)
    print("F1 score:", round(f1, 3))



def train_lgbm_model(df, test_size=0.25, max_depth=3, n_estimators=100):
    """
    A function to train a Light GBM model.
    Args:
        df: Input data, which needs to be split into train and test
        test_size: Size of the test set
        max_depth: Maximum depth of the Decision Tree
        n_estimators: Number of trees
        enable_categorical: Whether to enable categorical features
    Returns:
        None
    """

    # Libraries
    import pandas as pd
    import lightgbm as lgb
    from sklearn import metrics
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder

    # Convert categorical columns to numerical values
    for column in cars_df.select_dtypes(include='object').columns:
        label_encoder = LabelEncoder()
        cars_df[column] = label_encoder.fit_transform(cars_df[column])

    # Train, test split
    print("Generating train and test data...")
    x = cars_df[["buying", "maint", "doors", "persons", "lug_boot", "safety"]]
    y = cars_df[["class"]]
    X_train, X_test, y_train, y_test = train_test_split(
        x, y, test_size=test_size, shuffle=True
    )

    # Create a label encoder for the target variable
    label_encoder = LabelEncoder()
    y_train = label_encoder.fit_transform(y_train)
    y_test = label_encoder.transform(y_test)

    # Create a LightGBM model
    model = lgb.LGBMClassifier(objective='binary', max_depth=max_depth, n_estimators=n_estimators)

    # Train the model on the train set
    print("Training model...")
    model.fit(X_train, y_train)

    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print("Accuracy:", round(accuracy, 3))
    precision = metrics.precision_score(y_test, y_pred)
    print("Precision:", round(precision, 3))
    recall = metrics.recall_score(y_test, y_pred)
    print("Recall:", round(recall, 3))
    f1 = metrics.f1_score(y_test, y_pred)
    print("F1 score:", round(f1, 3))

### Initialize Vertex AI SDK for Python and set seed for reproducibility

Initialize the Vertex AI SDK for Python for your project and set seed to guarantee reproducibility.

In [ ]:
vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)
set_seed(8)

## Model experimentation using autologging with Vertex AI Experiments

Vertex AI Experiments Autologging allows you to run experiments and autologging parameters and metrics of different ML frameworks.

After initiating an Vertex AI Experiment, enable autologging using `vertex_ai.autolog()`.

There are two ways to use Autologging:

1.   *With automatic experiment run creation*
2.   *With user experiment run creation*

With *automatic experiment run creation*, you run an experiment. Vertex AI SDK automatically creates an experiment run by logging all paramenters and metrics in Vertex AI Experiments.

With *user experiment run creation*, you create an experiment using `vertex_ai.start_run(your-experiment-run-name)` and run the experiment. Then you get access to resulting paramentes and metrics after you end the experiment run with `vertex_ai.end_run()`


#### Create an experiment for tracking training parameters and metrics

To start, initiate an experiment using the `init()` method.

Because some model types like TensorFlow result in autologging time series metrics, you need to create a TensorBoard instance.

To create a TensorBoard instance, you can use `vertex_ai.Tensorboard.create()`.


 ⚠⚠⚠ **Warning** Currently, Vertex AI TensorBoard charges a monthly fee of 300 USD per unique active user. Beginning in August 2023, Vertex AI TensorBoard will change from a per-user monthly license of 300 USD per month to 10 USD per GiB per month for storage of logs and metrics. This means there are no more subscription fees, and you will only pay for the storage you've used. See the [Vertex AI TensorBoard: Delete Outdated TensorBoard Experiments tutorial](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/experiments/delete_outdated_tensorboard_experiments.ipynb) for how to manage storage. More info on Tensorboard [here](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview).


In [ ]:
AUTOLOGGED_EXPERIMENT_NAME = "my-experiment"  # @param {type:"string"}

In [ ]:
EXPERIMENT_TENSORBOARD = vertex_ai.Tensorboard.create()

In [ ]:
vertex_ai.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=BUCKET_URI,
    experiment=AUTOLOGGED_EXPERIMENT_NAME,
    experiment_tensorboard=EXPERIMENT_TENSORBOARD,
    experiment_description="autolog-experiment-with-automatic-run",
)

#### Autologging an experiment with automatic experiment run creation

In this section, Vertex AI SDK automatically creates an experiment run for you by logging all paramenters and training and post-training metrics in Vertex AI Experiments.


##### Enable autologging

First, enable autologging using `vertex_ai.autolog()` method.

After calling `vertex_ai.autolog()`, any metrics and parameters from
model training calls with supported ML frameworks will be automatically
logged to Vertex Experiments.

In [ ]:
vertex_ai.autolog()

#### Experiment 1 - Sklearn model

Next, define your baseline model by running a Sklearn model experiment.

In [ ]:
train_sklearn_model(cars_df, test_size=0.2, max_depth=5)

##### Get the experiment results

Then, use the method `get_experiment_df()` to get the results of the experiment as a pandas dataframe.

Notice how all paramenters and metrics are logged in Vertex AI Experiments.

In particular, the `run_name` has been automatically assigned and the evaluation metrics you defined have been logged too.

In [ ]:
experiment_df = vertex_ai.get_experiment_df()
experiment_df = experiment_df.T
experiment_df

#### Experiment 2 - LGBM model.

In [ ]:
train_lgbm_model(cars_df, test_size=0.2, max_depth=3, n_estimators=50)

#### Experiment 3 - XGBoost model

Let's train 3 models, changing the parameters max_depth and n_estimators.

In [ ]:
train_xgboost_model(cars_df, test_size=0.2, max_depth=3, n_estimators=50)

In [ ]:
train_xgboost_model(cars_df, test_size=0.2, max_depth=5, n_estimators=100)

In [ ]:
train_xgboost_model(cars_df, test_size=0.2, max_depth=10, n_estimators=200)

#### Compare experiment results

In [ ]:
experiment_df = vertex_ai.get_experiment_df()
experiment_df.T